### **連接並mount 雲端硬碟**

In [1]:
#定義路徑
SYS_DIR = "/content/drive/My Drive/Colab Notebooks/RNN_Pytorch_Online_Shopping/"
trained_Model_Path = "trained_model"
trained_Model_Name = "RNN-model.pt"

In [2]:
#連接並mount自己的雲端硬碟(點選跑出網址複製最後的授權碼貼上並執行)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#根目錄 '/content/drive/My Drive/'

Mounted at /content/drive


In [3]:
import os
os.chdir(SYS_DIR)

In [4]:
!ls

'Copy of RNN.ipynb'   dict   emotionClassifier_lstm.ipynb


### **下載dataset**

In [5]:
import os
def fileExists(filepath):
  return  os.path.exists(filepath)
#我把資料集放到我的github
if fileExists(SYS_DIR+'dataset/online_shopping_10_cats.csv')!= True:
  !wget -P dataset/ https://raw.githubusercontent.com/Edward83528/pytorch_online_shopping_emotion/master/dataset/online_shopping_10_cats.csv

--2020-10-17 15:43:14--  https://raw.githubusercontent.com/Edward83528/pytorch_online_shopping_emotion/master/dataset/online_shopping_10_cats.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11299695 (11M) [text/plain]
Saving to: ‘dataset/online_shopping_10_cats.csv’

online_shopping_10_ 100%[===================>]  10.78M  24.8MB/s    in 0.4s    

2020-10-17 15:43:15 (24.8 MB/s) - ‘dataset/online_shopping_10_cats.csv’ saved [11299695/11299695]



In [6]:
#引入套件
import pandas as pd
import numpy as np

In [7]:
#加載資料集

pd_all = pd.read_csv(SYS_DIR + 'dataset/online_shopping_10_cats.csv')

print('評論數目（全）：%d' % pd_all.shape[0])
print('評論數目（正向）：%d' % pd_all[pd_all.label==1].shape[0])
print('評論數目（負向）：%d' % pd_all[pd_all.label==0].shape[0])
print(type(pd_all))

評論數目（全）：62774
評論數目（正向）：31728
評論數目（負向）：31046
<class 'pandas.core.frame.DataFrame'>


In [8]:
#隨機抽樣20筆
pd_all.sample(20)

,cat,label,review
55646,酒店,1,7月份住了2天。是我住过最好的4星级宾馆之一。交通非常方便，离榆次老城非常近，走路5分钟可到...
52915,酒店,1,"房间内设施还不错,不过浴室的淋浴喷头不好用,水量不可调节,出水较散,有些不象四星级的标准.周..."
50729,计算机,1,总之第一句话就是便宜~让同事帮忙装了xp系统，运行起来还是不错的~
33056,洗发水,0,这次，京东做事让我很失望。买的洗发露，包装得不好，送到打开流了很多出来。。。无语了。
16320,水果,1,苹果不错，送货很快，京东值得信赖
39406,衣服,1,货已收到，衣服还没来得及穿，看着样式还不错
3826,书籍,0,2009.03.08订的货，03.17才收到。不是很快呀。
11198,平板,0,微信，第三方视频聊天前置摄像头画面黑的好厉害，怎么样设置都不能正常，手机本身没有问题，但是第...
58458,酒店,0,优点：交通便利缺点：不符合我心中的四星1.酒店的泳池还不如比它小的酒店漂亮。2.在三亚湾又不...
20863,水果,1,好好好好，苹果很好吃，下次还来买


In [9]:
#構造平衡語料
pd_positive = pd_all[pd_all.label==1]
pd_negative = pd_all[pd_all.label==0]

def get_balance_corpus(corpus_size, corpus_pos, corpus_neg):
    sample_size = corpus_size // 2
    pd_corpus_balance = pd.concat([corpus_pos.sample(sample_size, replace=corpus_pos.shape[0]<sample_size), \
                                   corpus_neg.sample(sample_size, replace=corpus_neg.shape[0]<sample_size)])
    
    print('評論數目(總體)：%d' % pd_corpus_balance.shape[0])
    print('評論數目(正向)：%d' % pd_corpus_balance[pd_corpus_balance.label==1].shape[0])
    print('評論數目(負向)：%d' % pd_corpus_balance[pd_corpus_balance.label==0].shape[0])    
    
    return pd_corpus_balance

In [10]:
pd_60000 = get_balance_corpus(60000, pd_positive, pd_negative)

評論數目(總體)：60000
評論數目(正向)：30000
評論數目(負向)：30000


In [11]:
#隨機抽樣20筆
pd_60000.sample(20)

,cat,label,review
38579,蒙牛,0,蒙牛伊利 写的吧，，，扯淡 哎
44181,衣服,0,什么商铺寄件烂衣服给我
57171,酒店,1,我们定的是豪华大床房，房间很漂亮也很干净，就是卫生间一洗澡就水漫金山。第一天去在修管道没热水...
21089,水果,1,好吃，比一般超市的好吃，脆甜。才出库的，冰冰凉凉的。
52691,计算机,0,外观一般。笨重！ 用了2天，感觉音质一般，没有介绍的那样出色。 触摸板离键盘近，打字挺容易碰...
31406,洗发水,0,这个差评是给快递的，现在想想还生气，打电话催我快下楼取快递说是还有其他客户等着取货，结果我挂...
60274,酒店,0,每次去开房间都要等1个小时，次序混乱，希望能改进服务！
52073,计算机,0,"只有安装了DOS系统,随机也没带系统安装盘.装XP时,显卡驱动没有,得问售后服务要,比较麻烦..."
61160,酒店,0,环境不错但是设施太旧，服务一般
39285,衣服,1,非常好的裤子 质量很好客服态度很好 很快就安排发货了 快递速度也挺不错的 五星好评啦


#數據集分割
##一般我們做機器學習會將數據分為訓練集和測試集，而在深度學習中，需要多輪訓練學習，每次的學習過程都包括訓練和驗證，最後再進行測試。所以需要將數據分成訓練、驗證和測試數據。

In [12]:
def split_csv(infile, trainfile, valtestfile, seed=999, ratio=0.2):	
    df = pd.read_csv(infile)
    df["review"] = df.review.str.replace("\n", " ")	
    idxs = np.arange(df.shape[0])	
    np.random.seed(seed)	
    np.random.shuffle(idxs)	
    val_size = int(len(idxs) * ratio)	
    df.iloc[idxs[:val_size], :].to_csv(valtestfile, index=False)	
    df.iloc[idxs[val_size:], :].to_csv(trainfile, index=False)
def split_dataFrame(df, trainfile, valtestfile, seed=999, ratio=0.2):	
    df["review"] = df.review.str.replace("\n", " ")	
    idxs = np.arange(df.shape[0])	
    np.random.seed(seed)	
    np.random.shuffle(idxs)	
    val_size = int(len(idxs) * ratio)	
    df.iloc[idxs[:val_size], :].to_csv(valtestfile, index=False)	
    df.iloc[idxs[val_size:], :].to_csv(trainfile, index=False)
#先將數據分為train.csv和test.csv	
split_dataFrame(df=pd_60000, 	
        trainfile=SYS_DIR + 'dataset/train.csv', 	
        valtestfile=SYS_DIR + 'dataset/test.csv', 	
        seed=999, 	
        ratio=0.2)
#再將train.csv分為dataset_train.csv和dataset_valid.csv
split_csv(infile=SYS_DIR + 'dataset/train.csv', 	
        trainfile=SYS_DIR + 'dataset/dataset_train.csv', 	
        valtestfile=SYS_DIR + 'dataset/dataset_valid.csv', 	
        seed=999, 	
        ratio=0.2)

In [13]:
#安裝結巴
!pip install jieba

In [14]:
# OpenCC 來達成繁簡轉換
! pip install opencc-python-reimplemented

     |████████████████████████████████| 491kB 3.4MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486137 sha256=2465260e1e9c3762175c4feacb5a63c060ee1cf84b2fc4b0dac69abd0e6bccd3
  Stored in directory: /root/.cache/pip/wheels/54/8a/0f/405db0ba3f02d9af8211422d3ae94c47b3296256f168ac1e68
Successfully built opencc-python-reimplemented


In [15]:
import jieba
import jieba.posseg as psg
import codecs

#利用OpenCC將簡體中文轉繁體中文
from opencc import OpenCC
cc = OpenCC('s2t') #s2t: 簡體中文 -> 繁體中文

# jieba custom setting.
jieba.set_dictionary(SYS_DIR + 'dict/dict.txt.big.txt')


# load stopwords set 引入了停用詞(停用詞就是像英文中的 the,a,this，中文的你我他，與其他詞相比顯得不怎麼重要)
stopword_set = set()
with codecs.open(SYS_DIR + 'dict/stopwords.txt','r','utf-8') as stopwords:
    for stopword in stopwords:
        stopword_set.add(stopword.strip('\n'))
	
#中文斷詞
def tokenize(sentence):
    converted = cc.convert(sentence) #簡>繁
    words = jieba.posseg.cut(converted)
    words = [w.word for w in words if w.word not in stopword_set ]
    #print(words)
    return words
	
print(tokenize('傅达仁今将运行安乐死，却突然爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。'))

Building prefix dict from /content/drive/My Drive/Colab Notebooks/RNN_Pytorch_Online_Shopping/dict/dict.txt.big.txt ...
Dumping model to file cache /tmp/jieba.u129ee828390e518a65fdc8e349e93ce4.cache
Loading model cost 1.956 seconds.
Prefix dict has been built successfully.


['傅達仁', '今', '運行', '安樂死', '卻', '突然', '爆出', '20', '年前', '遭緯', '體育臺', '封殺', '懂', '哪裏', '得罪', '電視臺']


## 安裝pytorch

In [16]:
!pip install torch torchvision torchtext

import torch
from torch import nn
from torchtext import data, datasets

#設置固定生成隨機數的種子，使得每次運行文件時生成的隨機數相同
if torch.cuda.is_available():
    print("gpu cuda is available!")
    torch.cuda.manual_seed(1000)
else:
    print("cuda is not available! cpu is available!")
    torch.manual_seed(1000)

gpu cuda is available!


In [17]:
CAT = data.Field(sequential=True, fix_length=20)
LABEL = data.LabelField(dtype = torch.float)	
TEXT = data.Field(tokenize = tokenize,#斷詞function 	
                      lower=True,	#是否將數據轉小寫
                      fix_length=100,	#每條數據的長度
                      stop_words=None)
train_data, valid_data, test_data = data.TabularDataset.splits(path=SYS_DIR+'dataset/', #數據所在文件夾
                                  train='dataset_train.csv', 	
                                  validation='dataset_valid.csv',	
                                  test = 'test.csv',	
                                  format='csv',
                                  skip_header=True,	
                                  fields = [('cat',CAT),('label', LABEL),('review', TEXT)])

In [18]:
print('len of train_data:', len(train_data))
print('len of test data:', len(test_data))
print('len of valid_data:', len(valid_data))

len of train_data: 38400
len of test data: 12000
len of valid_data: 9600


In [19]:
print(train_data.examples[18].review)
print(train_data.examples[18].label)

['最後', '兩件', '搞錯', '一件', '那件', '第一件', '薄', '鬆懈', '舒適感', '極差']
0


In [20]:
# 建立詞典
#TEXT.build_vocab(train_data, vectors="glove.6B.100d")#可以使用預訓練word vector
TEXT.build_vocab(train_data, max_size=10000)
LABEL.build_vocab(train_data)
CAT.build_vocab(train_data)

print(TEXT.vocab.freqs.most_common(20)) # 資料集裡最常出現的20個詞
print(TEXT.vocab.itos[:10]) #列表 index to word
print(TEXT.vocab.stoi) #字典 word to index

batch_size = 30
device = torch.device('cuda')
#Iterator是torchtext到模型的輸出
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),
    batch_size = batch_size,
    sort_key=lambda x: len(x.review),
    device=device
)

[(' ', 18958), ('好', 10736), ('買', 9967), ('不錯', 9301), ('酒店', 6798), ('京東', 5421), ('房間', 4837), ('說', 4824), ('一個', 3952), ('感覺', 3848), ('差', 3630), ('人', 3622), ('太', 3569), ('質量', 3155), ('服務', 3034), ('蘋果', 2986), ('比較', 2922), ('小', 2741), ('喜歡', 2701), ('東西', 2680)]
['<unk>', '<pad>', ' ', '好', '買', '不錯', '酒店', '京東', '房間', '說']
defaultdict(<function _default_unk_index at 0x7fc84fd3bd08>, {'<unk>': 0, '<pad>': 1, ' ': 2, '好': 3, '買': 4, '不錯': 5, '酒店': 6, '京東': 7, '房間': 8, '說': 9, '一個': 10, '感覺': 11, '差': 12, '人': 13, '太': 14, '質量': 15, '服務': 16, '蘋果': 17, '比較': 18, '小': 19, '喜歡': 20, '東西': 21, '住': 22, '價格': 23, '書': 24, '知道': 25, '後': 26, '滿意': 27, '有點': 28, '問題': 29, '吃': 30, '不好': 31, '快遞': 32, '挺': 33, '收到': 34, '手機': 35, '再': 36, '褲子': 37, '入住': 38, '真的': 39, '着': 40, '一直': 41, '方便': 42, '物流': 43, '包裝': 44, '蒙牛': 45, '中': 46, '送': 47, '以後': 48, '一次': 49, '特別': 50, '客服': 51, '點': 52, '穿': 53, '舒服': 54, '味道': 55, '一點': 56, '現在': 57, '因爲': 58, '屏幕': 59, '想': 60, '裏': 61, '已經':

In [21]:
#創⼀個資料夾存模型
if not os.path.isdir(trained_Model_Path):
    os.mkdir(trained_Model_Path)

In [22]:
vocab_size=len(TEXT.vocab) #詞典大小
embedding_dim=100 # 詞向量維度
hidden_dim=100 # 隱藏層維度
output_dim=1 # 輸出層

In [23]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim , output_dim):
      
        super(RNN, self).__init__()
        
        # nn.Embedding 可以幫我們建立好字典中每個字對應的 vector
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, 
                           bidirectional=True, dropout=0)
        # [100*2] => [1]
        # Fully-connected layer，把 hidden state 線性轉換成 output
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(0)  #防止過擬合 (overfitting)
        
        
    def forward(self, x):
        """
        x: [seq_len, b] vs [b, 3, 28, 28]
        """
        # [seq, b, 1] => [seq, b, 100]
        embedding = self.dropout(self.embedding(x))
        
        # output: [seq, b, hid_dim*2]
        # hidden/h: [num_layers*2, b, hid_dim]
        # cell/c: [num_layers*2, b, hid_di]
        output, (hidden, cell) = self.rnn(embedding)
        
        # [num_layers*2, b, hid_dim] => 2 of [b, hid_dim] => [b, hid_dim*2]
        hidden = torch.cat([hidden[-2], hidden[-1]], dim=1)
        
        # [b, hid_dim*2] => [b, 1]
        hidden = self.dropout(hidden)
        # 線性轉換至 output
        out = self.fc(hidden)
        
        return out

In [24]:
rnn = RNN(vocab_size, embedding_dim, hidden_dim , output_dim)

pretrained_embedding = TEXT.vocab.vectors
#print('pretrained_embedding:', pretrained_embedding.shape)
#rnn.embedding.weight.data.copy_(pretrained_embedding)
#print('embedding layer inited.')

#優化器
optimizer = torch.optim.Adam(rnn.parameters(), lr=1e-3)
loss_func = torch.nn.BCEWithLogitsLoss().to(device) #因為我們輸出只有1個，所以選用binary cross entropy
#用gpu
rnn.to(device)

RNN(
  (embedding): Embedding(10002, 100)
  (rnn): LSTM(100, 100, num_layers=2, bidirectional=True)
  (fc): Linear(in_features=200, out_features=1, bias=True)
  (dropout): Dropout(p=0, inplace=False)
)

In [25]:

def binary_acc(preds, y):
    """
    準確率
    """
    preds = torch.round(torch.sigmoid(preds))
    correct = torch.eq(preds, y).float()
    acc = correct.sum() / len(correct)
    return acc
  
def train(rnn, epoch, iterator, optimizer, loss_func):

    rnn.train() # 訓練模式

    avg_acc = []
    TrainAcc = 0.0
    TrainLoss = 0.0
    
    for step,batch in enumerate(iterator):

        optimizer.zero_grad() #優化器優化之前須將梯度歸零

        # [seq, b] => [b, 1] => [b]
        pred = rnn(batch.review).squeeze(1)
        
        acc = binary_acc(pred, batch.label).item() # 取得正確率
        TrainAcc=TrainAcc+acc
        avg_acc.append(acc)

        loss = loss_func(pred, batch.label) # loss計算
        TrainLoss = TrainLoss + loss
        
        
        loss.backward() #反向傳遞
        optimizer.step() #通過梯度做參數更新(更新權重)
        
        #if step%20 == 0:print('train loss: %.4f, train acc: %.2f' %(loss, acc))
            
    TrainLoss = TrainLoss / (step+1) # epoch loss
    TrainAcc = TrainAcc / (step+1) # epoch acc    
    avg_acc = np.array(avg_acc).mean()
    print('epoch : %d ,TrainLoss: %f , TrainAcc acc: %f , avg TrainAcc acc: %f ' %(epoch+1,round(TrainLoss.item(),3), round(TrainAcc,3), avg_acc ))

    if avg_acc > 0.9:
      torch.save(rnn, SYS_DIR+trained_Model_Path+'/'+trained_Model_Name) #存模型
      #torch.save(rnn.state_dict(), SYS_DIR+trained_Model_Path+'/'+trained_Model_Name) #只存權重
      print('save done')
    
    
    
def eval(rnn, epoch , iterator, loss_func):

    rnn.eval() # 評估模式
    
    avg_acc = []
    TestAcc = 0.0
    TestLoss = 0.0    
    
    with torch.no_grad():
        for step,batch in enumerate(iterator):
            # [b, 1] => [b]
            pred = rnn(batch.review).squeeze(1)

            acc = binary_acc(pred, batch.label).item() # 取得正確率
            TestAcc = TestAcc + acc
            avg_acc.append(acc)

            loss = loss_func(pred, batch.label) # loss計算
            TestLoss = TestLoss + loss
            
            #if step%20 == 0:print('test loss: %.4f, test acc: %.2f' %(loss, acc))

        TestLoss = TestLoss / (step+1)
        TestAcc = TestAcc / (step+1)
        avg_acc = np.array(avg_acc).mean()
        print('epoch : %d ,TestLoss: %f , TestAcc acc: %f , avg TestAcc acc: %f ' %(epoch+1,round(TestLoss.item(),3), round(TestAcc,3), avg_acc ))

def predict_sentiment(net,sentence):
    net.eval()
    # 分詞
    tokenized = tokenize(sentence)
    # sentence 的索引
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]

    tensor = torch.LongTensor(indexed).to(device)  # seq_len
    tensor = tensor.unsqueeze(1)   # seq_len * batch_size (1)

    # tensor寫text一樣的tensor
    prediction = torch.sigmoid(net(tensor))

    return prediction.item()    


In [26]:
# 開始訓練
print('start train')
for epoch in range(10):
  train(rnn, epoch , train_iterator, optimizer, loss_func)
  eval(rnn, epoch , test_iterator, loss_func)
print('end train')
torch.save(rnn, SYS_DIR+trained_Model_Path+'/'+trained_Model_Name) #存模型

start train
epoch : 1 ,TrainLoss: 0.375000 , TrainAcc acc: 0.832000 , avg TrainAcc acc: 0.832214 
epoch : 1 ,TestLoss: 0.278000 , TestAcc acc: 0.887000 , avg TestAcc acc: 0.887333 
epoch : 2 ,TrainLoss: 0.242000 , TrainAcc acc: 0.905000 , avg TrainAcc acc: 0.905052 
epoch : 2 ,TestLoss: 0.255000 , TestAcc acc: 0.903000 , avg TestAcc acc: 0.903000 
epoch : 3 ,TrainLoss: 0.209000 , TrainAcc acc: 0.920000 , avg TrainAcc acc: 0.920130 
epoch : 3 ,TestLoss: 0.476000 , TestAcc acc: 0.754000 , avg TestAcc acc: 0.753833 
epoch : 4 ,TrainLoss: 0.180000 , TrainAcc acc: 0.933000 , avg TrainAcc acc: 0.933229 
epoch : 4 ,TestLoss: 0.256000 , TestAcc acc: 0.902000 , avg TestAcc acc: 0.901583 
epoch : 5 ,TrainLoss: 0.136000 , TrainAcc acc: 0.953000 , avg TrainAcc acc: 0.953021 
epoch : 5 ,TestLoss: 0.266000 , TestAcc acc: 0.905000 , avg TestAcc acc: 0.904833 
epoch : 6 ,TrainLoss: 0.106000 , TrainAcc acc: 0.964000 , avg TrainAcc acc: 0.963854 
epoch : 6 ,TestLoss: 0.301000 , TestAcc acc: 0.903000 , a

## 讀取模型並預測

In [31]:
net = torch.load(SYS_DIR+trained_Model_Path+'/'+trained_Model_Name)

while True:
  user_key = input("輸入評價:")
  if(user_key=='end'):
    break
  score=predict_sentiment(net,user_key)
  if score>0.5:
    print(score,':正面')
  else:
    print(score,'負面')

輸入評價:包装太差了，货到时两瓶洗发水已经全部漏出来了，瓶子里面一点都没剩
0.0018897760892286897 負面
輸入評價:包装太差了，货到时两瓶洗发水已经全部漏出来了，瓶子里面一点都没剩
0.0018897760892286897 負面
輸入評價:非常好的裤子 质量很好客服态度很好 很快就安排发货了 快递速度也挺不错的 五星好评啦
0.9989175796508789 :正面
輸入評價:这酒店垃圾到了极点 设施超级垃圾 而且服务态度极差 已投诉主管部门
4.296930455893744e-07 負面
輸入評價:上餐速度非常快,這家店不錯
0.9974379539489746 :正面
輸入評價:上餐速度非常慢,這家店很爛
2.6171990612056106e-05 負面
輸入評價:我今天心情非常好
0.5912814736366272 :正面
輸入評價:我今天心情不好
0.04821079596877098 負面
輸入評價:end
